In [ ]:

import yaml
import pandas as pd
import requests
import io
import os

# create folders to save data (git does not track empty directories)

os.makedirs('data/', exist_ok=True)
os.makedirs('data/processed/', exist_ok=True)  
os.makedirs('data/raw/', exist_ok=True)  

# load login credentials

with open("config/login.yaml", "r", encoding="utf-8") as file:
    conf = yaml.safe_load(file)

config = {
    "path": conf["access_path"],
    "format": conf["format"]
}

login = {
    "username": conf["username"],
    "password": conf["password"]
}


## Microcensus data

The microcensus is a survey conducted by the national statistical office that collects data about a fraction of the total national population. Let us look at which data tables exist for the microcensus:

In [ ]:
def get_table_list(selection, parameters, config):
    parameters["selection"] = selection
    url = f"{config['path']}/catalogue/tables"
    return requests.get(url, params=parameters).json()

In [ ]:
relevant_tables = get_table_list("12211*", login, config)
relevant_tables

Let us look at the data for a particular table, e.g. `12211-0003` looks potentially interesting because it has info on income. 

In [ ]:

def get_data(table, parameters, config):
    csv_path = f"data/raw/tablefile_{table}.csv"
    if os.path.isfile(csv_path):
            data = pd.read_csv(csv_path)
    else:
            data = get_data_from_api(table, parameters, config)
    return data

        
def get_data_from_api(table, parameters, config):
    print(f"Requesting table data for table '{table}'")
    parameters["name"] = table

    # use "ffcsv" format because it is consistent across tables
    # and over time
    parameters["format"] = config["format"]
    url = f"{config['path']}/data/tablefile"
    
    response = requests.get(url, params=parameters)
    if response.ok:
        data = pd.read_csv(
            io.StringIO(response.content.decode("utf-8")),
            sep = ";",
            decimal = ",",
            na_values = ["-"],
        )
        data.to_csv(f"data/raw/tablefile_{table}.csv", index=False)
        return data
    else:
        return response


In [ ]:
my_table = "12211-0003"

data = get_data(my_table, login, config)


In [ ]:
data.head()

In [ ]:
my_data.columns